### 学习笔记
代码借鉴于公众号机器学习实验室  
主要学习python内置模块collections的相关内容  
不足：代码未能实现书中所建议的数据结构kd搜索树。

In [6]:
import numpy as np
from collections import Counter
import random
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.utils import shuffle

In [8]:
iris = load_iris()
X,Y = iris.data,iris.target
X[0]

array([5.1, 3.5, 1.4, 0.2])

In [9]:
X,Y = shuffle(iris.data,iris.target,random_state=13)
X[0]

array([5.7, 2.8, 4.5, 1.3])

In [20]:
X = X.astype(np.float32)
offset = int(X.shape[0]*0.7)
# 训练集
X_train, Y_train = X[:offset], Y[:offset]
# 测试集
X_test, Y_test = X[offset:], Y[offset:]
Y_train = Y_train.reshape((-1,1))
Y_test = Y_test.reshape((-1,1))

print('X_train ', X_train.shape)
print('X_test ', X_test.shape)
print('Y_train ', Y_train.shape)
print('Y_test ', Y_test.shape)

X_train  (105, 4)
X_test  (45, 4)
Y_train  (105, 1)
Y_test  (45, 1)


In [55]:
# 定义L2距离度量函数
def compute_distances(X, X_train):
    num_test = X.shape[0]
    num_train = X_train.shape[0]
    dists = np.zeros((num_test, num_train)) 

    M = np.dot(X, X_train.T)
    te = np.square(X).sum(axis=1)
    tr = np.square(X_train).sum(axis=1)
    dists = np.sqrt(-2 * M + tr + np.matrix(te).T)    
    return dists

In [37]:
x_01 = X_test[0,:]
print(type(x_01))
print(x_01.shape)
x_01

<class 'numpy.ndarray'>
(4,)


array([5.5, 3.5, 1.3, 0.2], dtype=float32)

In [38]:
x_02 = X_test[0,:].reshape((1,-1))
print(type(x_02))
print(x_02.shape)
x_02

<class 'numpy.ndarray'>
(1, 4)


array([[5.5, 3.5, 1.3, 0.2]], dtype=float32)

In [56]:
dists = compute_distances(X_test,X_train)
print(dists.shape)
dists

(45, 105)


matrix([[3.4612148 , 2.6210682 , 0.5291419 , ..., 0.60827225, 0.5477184 ,
         3.2046835 ],
        [0.8888221 , 0.9486861 , 3.109662  , ..., 2.9832857 , 3.1796227 ,
         0.73484755],
        [0.45824826, 0.7071014 , 3.1032243 , ..., 2.918904  , 3.163857  ,
         0.46903068],
        ...,
        [3.5142565 , 2.6720777 , 0.41230372, ..., 0.3999998 , 0.38729537,
         3.2710843 ],
        [0.73484755, 1.0630169 , 3.3136082 , ..., 3.093541  , 3.3674917 ,
         0.8306547 ],
        [0.6244963 , 0.42425993, 2.8089156 , ..., 2.6608274 , 2.8757615 ,
         0.37415472]], dtype=float32)

In [63]:
dists = compute_distances(x_02,X_train)
print(dists.shape)
dists

(1, 105)


matrix([[3.4612136 , 2.6210682 , 0.5291491 , 4.2497063 , 4.8155994 ,
         0.70710945, 5.704384  , 4.09878   , 0.5830986 , 3.3793492 ,
         3.9127996 , 4.1545153 , 3.5916576 , 0.80622673, 4.4564567 ,
         0.8602357 , 4.016217  , 5.049752  , 3.597221  , 1.2569782 ,
         3.5411859 , 0.96436524, 0.42425993, 5.479964  , 3.491418  ,
         5.1117506 , 0.5916099 , 6.4358373 , 3.7040515 , 4.877499  ,
         4.601088  , 0.70710677, 0.31622535, 4.579301  , 4.268489  ,
         3.325658  , 0.6708181 , 3.491418  , 3.7563276 , 0.87177664,
         0.36055133, 4.593473  , 3.1654365 , 5.256425  , 4.332436  ,
         5.588381  , 0.4582524 , 2.5159488 , 1.1401751 , 3.7510004 ,
         0.54772186, 3.114482  , 3.8807218 , 5.229723  , 0.75498074,
         2.87576   , 0.90554047, 2.9291635 , 4.509989  , 5.297169  ,
         1.0049887 , 0.70710677, 4.6957417 , 3.768288  , 5.5821137 ,
         4.6314144 , 0.5196157 , 3.721559  , 2.8248892 , 4.1988096 ,
         0.34640312, 0.42425993, 0

In [75]:
# 定义多数表决函数
def predict_labels(y_train, dists, k):
    num_test = dists.shape[0]
    y_pred = np.zeros(num_test)    
    for i in range(num_test):

        closest_y = []
        # 注意 argsort 函数的用法
        labels = y_train[np.argsort(dists[i, :])].flatten()
        closest_y = labels[0:k]

        c = Counter(closest_y)
        # print(c)
        y_pred[i] = c.most_common(1)[0][0]    
    return y_pred

In [84]:
def accuracy(X_test,Y_test,X_train,Y_train,k):
    dists = compute_distances(X_test, X_train)
    Y_pred = predict_labels(Y_train, dists, k)
    num_correct = np.sum(Y_pred == Y_test.reshape((1,-1)))
    accuracy = float(num_correct) / X_test.shape[0]
    return ('Got %d / %d correct => accuracy: %f' % (num_correct, X_test.shape[0], accuracy))

In [87]:
accuracy(X_test,Y_test,X_train,Y_train,1)

'Got 44 / 45 correct => accuracy: 0.977778'

In [85]:
accuracy(X_test,Y_test,X_train,Y_train,5)

'Got 44 / 45 correct => accuracy: 0.977778'

In [86]:
accuracy(X_test,Y_test,X_train,Y_train,10)

'Got 44 / 45 correct => accuracy: 0.977778'

In [88]:
accuracy(X_test,Y_test,X_train,Y_train,20)

'Got 43 / 45 correct => accuracy: 0.955556'

In [72]:
# 检验测试数据集的准确率
Y_test_pred = predict_labels(Y_train,dists, 99)

Y_test_pred = Y_test_pred.reshape((-1, 1))

num_correct = np.sum(Y_test_pred == Y_test[0,:].reshape((1,-1)))

accuracy = float(num_correct) / x_02.shape[0]

print('Got %d / %d correct => accuracy: %f' % (num_correct, x_02.shape[0], accuracy))

Counter({0: 35, 1: 33, 2: 31})
Got 1 / 1 correct => accuracy: 0.250000
